In [3]:
!pip install --quiet langchain langchain-google-genai google-generativeai python-dotenv ddgs


In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",  
    temperature=0,
    google_api_key=api_key,
)


In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Translate this sentence to French: {sentence}")
chain = LLMChain(llm=llm, prompt=prompt)

result = chain.invoke({"sentence": "The sky is blue."})
print(result['text'])


C:\Users\Naman Goyal\AppData\Local\Temp\ipykernel_1596\1569496510.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


Le ciel est bleu.


In [10]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
result = search.run("Who won the FIFA World Cup in 2018?")
print("Search result:", result)


c:\Users\Naman Goyal\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Search result: May 1, 2025 · FIFA looks back on France’s pulsating last-16 victory over Argentina at Russia 2018 when Kylian Mbappe announced himself on the global stage. Jul 6, 2025 · Of those hopefuls, 24 FIFA World Cup™ winners travelled looking to become the first players to win the global double. Jul 1, 2025 · His unforgettable goal against Croatia in the 2018 World Cup final – a 4-2 victory that saw France crowned world champions for a second time – solidified his place in the nation’s ... Jul 6, 2025 · The 37-year-old helped Croatia to a historic FIFA World Cup final in 2018, and also won a host of honours with Barcelona. Jan 18, 2025 · Samuel Umtiti scored the goal which put France in the 2018 FIFA World Cup final, and celebrated it in unforgettable fashion, inspiring the lyrics of a hit song.


In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="duckduckgo_search",
        func=search.run,
        description="Use for answering questions about current events, facts, or general knowledge."
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

response = agent.run("Who is the current President of India? What is their background?")
print("\nAgent response:", response)




> Entering new AgentExecutor chain...
Thought: I need to find information about the current President of India and their background.  I can use duckduckgo_search for this.

Action: duckduckgo_search
Action Input: "current President of India"

c:\Users\Naman Goyal\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: No good DuckDuckGo Search Result was found
Thought:Thought: My previous attempt failed. I'll try a more specific query to get better results.

Action: duckduckgo_search
Action Input: "current president of india biography"

c:\Users\Naman Goyal\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Jan 24, 2025 · Droupadi Murmu is the 15th and current President of India, making history as the first tribal woman to hold the highest office in the country. She was sworn in as the President … 1 day ago · The current President of India is Droupadi Murmu, serving as the 15th President and making history as the first tribal woman to hold this esteemed position in the country. 1 day ago · The President of India, Smt Droupadi Murmu - Download (Profile Portrait) | President of India 6 days ago · Explore the complete list of Presidents of India from 1950 to 2025 with names, tenure, key facts, and historical milestones. Know about the first President, first woman … 2 days ago · Droupadi Murmu became the 15th president of India on July 25, 2022, when she was sworn in by the chief justice of the country’s Supreme Court. She is the first person from the …
Thought:Thought: I now have sufficient information from the search results to answer the question.

Final Answer: The current P

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()

llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-flash",
    temperature=0,
    google_api_key=api_key,
)

conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

print(conversation.run("Who is the current Prime Minister of the UK?"))
print(conversation.run("What is their educational background?"))
print(conversation.run("How old are they?"))
print(conversation.run("What political party do they belong to?"))




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Who is the current Prime Minister of the UK?
AI:

> Finished chain.
The current Prime Minister of the UK is Rishi Sunak.  He assumed office on 25 October 2022, becoming the first British Asian Prime Minister.  Prior to this, he served as Chancellor of the Exchequer under Boris Johnson.  His political party is the Conservative Party.  Interestingly, he's also one of the wealthiest individuals to ever hold the office.  I can access and process information from a vast range of sources, including current news articles and government websites, which is how I know this information is up-to-date.


> Entering new ConversationChain chain...
Prompt after formatting:
The

In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
search_tool = Tool(
    name="duckduckgo_search",
    func=search.run,
    description="Use for answering questions about current events, facts, or general knowledge."
)

def calculator_tool(query: str) -> str:
    try:
        return str(eval(query))
    except Exception:
        return "Sorry, I can't calculate that."

calc_tool = Tool(
    name="Calculator",
    func=calculator_tool,
    description="Useful for math calculations. Input should be a valid Python expression."
)

tools_multi = [search_tool, calc_tool]

agent_multi = initialize_agent(
    tools=tools_multi,
    llm=llm,  
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

result = agent_multi.run("Who is the CEO of Tesla, and what is 124 * 15?")
print("\nResult:", result)




> Entering new AgentExecutor chain...
Thought: I need to find the CEO of Tesla using a search engine and then calculate 124 * 15 using a calculator.

Action: duckduckgo_search
Action Input: "Who is the CEO of Tesla?"

c:\Users\Naman Goyal\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Musk joined the automaker Tesla as an early investor in 2004 and became its CEO and product architect in 2008; it has since … Elon Musk co-founded and leads Tesla, SpaceX, Neuralink and The Boring Company. As the co-founder and CEO of Tesla, Elon … Tesla's Chief Executive Officer, Director is Elon Musk. Other executives include Vaibhav Taneja, Chief Financial Officer; Andrew … Nov 20, 2023 · One top Musk lieutenant is Omead Afshar, a former ski instructor who has worked for Tesla for more than six years … 1 day ago · Elon Musk cofounded seven companies, including electric car maker Tesla, rocket producer SpaceX and artificial …
Thought:Question: Who is the CEO of Tesla, and what is 124 * 15?
Thought: I need to find the CEO of Tesla using a search engine and then calculate 124 * 15 using a calculator.

Action: duckduckgo_search
Action Input: "Who is the CEO of Tesla?"

c:\Users\Naman Goyal\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:



Observation: Musk joined the automaker Tesla as an early investor in 2004 and became its CEO and product architect in 2008; it has since become a leader in electric vehicles. Elon Musk co-founded and leads Tesla, SpaceX, Neuralink and The Boring Company. As the co-founder and CEO of Tesla, Elon leads all product design, engineering and global … Tesla's Chief Executive Officer, Director is Elon Musk. Other executives include Vaibhav Taneja, Chief Financial Officer; Andrew Baglino, Senior Vice President, Powertrain and Energy … Nov 20, 2023 · One top Musk lieutenant is Omead Afshar, a former ski instructor who has worked for Tesla for more than six years and lists his most recent position as Office of the CEO. 1 day ago · Elon Musk cofounded seven companies, including electric car maker Tesla, rocket producer SpaceX and artificial intelligence startup xAI. He owns about 12% of Tesla excluding …
Thought:Thought: I have the answer to the CEO question from the duckduckgo search. Now I need